# Extending ITK with your own module

## Why?

* New algorithm
* Easy integration in ITK
* Reproducible science
* Easy Python wrapping

## Requirements

* CMake
* C++ development environment

## Why use C++

* C++ allows the developer to have full access to ITK
    * Memory management
    * Thread management
* Any dimension and pixel type supported.
* Key element missing in Python: Iterators.

## Iterators

* An iterator is a fast and safe way to access an entire range of memory.
* It can be used to process an entire image.

## ITK Modules

* ITK is organized in modules
* Each module typically contains filters that are related (e.g. Segmentation, Registration, Filtering)

## What are remote modules

* Remote modules are an easy way to develop new ITK filters and integrate them in ITK.
* Allows the developer to easily publish and share their new algorithms.
* ITK continuously contains the best available algorithms.
* Remote modules are separate project from ITK. The developer(s) is (are) responsible of maintaining the project.
* Remote modules integrate one or more filters implemented in C++ and their corresponding Python bindings.

## Examples

* ITKTextureFeatures
* ITKUltrasound
* ITKIsotropicWavelets

## ITKTextureFeatures

### Two filters:
* itkCoocurrenceTextureFeaturesImageFilter: computes textural features based on intensity-based co-occurrence matrices in the image.
* itkRunLengthTextureFeaturesImageFilter: computes textural features based on equally valued intensity clusters of different sizes or run lengths in the image.

<div style="width: 100%; display: table;">
    <div style="display: table-row">
        <div style="width: 350px; display: table-cell;">
            <figure>
                <img src="data/CBCT-TextureInput.png">
                <figcaption>Fig1. - CBCT of the test condyle</figcaption>
            </figure> 
        </div>
        <div style="width: 350px; display: table-cell;">
            <figure>
                <img src="data/CBCT-TextureRunLengthNonUniformity.png">
                <figcaption>Fig2. - texture’s run length non uniformity</figcaption>
            </figure> 
        </div>
    </div>
</div>

- https://github.com/InsightSoftwareConsortium/ITKTextureFeatures
- http://www.insight-journal.org/browse/publication/985

```
python -m pip install itk-texturefeatures
```

##  ITKUltrasound

* High performance ultrasound image formation and analysis based on the Insight Toolkit (ITK).
* Introduction of an itk::StreamingResampleImageFilter.
<figure>
    <img src="data/Ultrasound-B-mode.png" width=600>
    <figcaption>Fig1. Large, scan converted B-Mode image of anechoic spheres in a tissue mimicking ultrasound phantom. The image was processed without swapping to disk.</figcaption>
</figure>

- https://github.com/KitwareMedical/ITKUltrasound
- http://www.insight-journal.org/browse/publication/722

```
python -m pip install itk-ultrasound
```

##  ITKIsotropicWavelets

* Multiresolution (MRA) analysis framework using isotropic and steerable wavelets in the frequency domain.
* Provides the backbone for state of the art filters for denoising, feature detection or phase analysis in N-dimension

- https://github.com/InsightSoftwareConsortium/ITKIsotropicWavelets
- http://www.insight-journal.org/browse/publication/986

```
python -m pip install itk-isotropicwavelets
```

## Creation of a remote module: Overview

1. The developer creates a new module containing new ITK filters.
    * The new module is its own independent GitHub project.
    * The new module can be easily be compiled and used in combination with ITK.
2. The developer writes an Insight Journal article
    * The module is more visible to the community.
    * An option can be added to ITK to compile the remote module as part of ITK.

## Creation of a remote module: details

* The template project source code is here:
        https://github.com/InsightSoftwareConsortium/ITKModuleTemplate

* Run the following commands:
        python -m pip install cookiecutter
        python -m cookiecutter gh:InsightSoftwareConsortium/ITKModuleTemplate

* Provide requested information.
        Answer the following questions (Pressing "Enter" will use the default option):
        full_name [Insight Software Consortium]:
        email [itk+community@discourse.itk.org]:
        github_username [itkrobot]:
        project_name [ITKModuleTemplate]: 
        module_name [ModuleTemplate]: 
        python_package_name [itk-moduletemplate]: 
        download_url [https://github.com/InsightSoftwareConsortium/ITKModuleTemplate]: 
        project_short_description [This is a template that serves as a starting point for a new module.]: 
        project_long_description [ITK is an open-source, cross-platform library that provides developers with an extensive suite of software tools for image analysis. Developed through extreme programming methodologies, ITK employs leading-edge algorithms for registering and segmenting multidimensional scientific images.]: 

##  New Module Content
<pre>
        (itk) fbudin:ITKModuleTemplate/ $ tree -a
        .
        ├── appveyor.yml
        ├── .circleci
        │   └── config.yml
        ├── CMakeLists.txt
        ├── CTestConfig.cmake
        ├── include
        │   ├── itkMinimalStandardRandomVariateGenerator.h
        │   ├── itkMyFilter.h
        │   ├── itkMyFilter.hxx
        │   ├── itkNormalDistributionImageSource.h
        │   └── itkNormalDistributionImageSource.hxx
        ├── itk-module.cmake
        ├── LICENSE
        ├── README.rst
        ├── setup.py
</pre>

<pre>
        ├── src
        │   ├── CMakeLists.txt
        │   └── itkMinimalStandardRandomVariateGenerator.cxx
        ├── test
        │   ├── Baseline
        │   │   ├── itkMyFilterTestOutput.mha.sha512
        │   │   └── itkNormalDistributionImageSourceTestOutput.mha.sha512
        │   ├── CMakeLists.txt
        │   ├── itkMinimalStandardRandomVariateGeneratorTest.cxx
        │   ├── itkMyFilterTest.cxx
        │   └── itkNormalDistributionImageSourceTest.cxx
        ├── .travis.yml
        └── wrapping
            ├── CMakeLists.txt
            ├── itkMinimalStandardRandomVariateGenerator.wrap
            └── itkNormalDistributionImageSource.wrap
</pre>

##  Directory structure

* `src`  and `include`: header files and source code
* `test`: unit tests
* `wrapping`: Required files to automatically create Python bindings.

## Filter code

<pre>
template< typename TInputImage, typename TOutputImage >
void
MyFilter< TInputImage, TOutputImage >
::DynamicThreadedGenerateData( const OutputRegionType & outputRegion)
{
  OutputImageType * output = this->GetOutput();
  const InputImageType * input = this->GetInput();
  using InputRegionType = typename InputImageType::RegionType;
  InputRegionType inputRegion = InputRegionType(outputRegion.GetSize());

  itk::ImageRegionConstIterator<InputImageType> in(input, inputRegion);
  itk::ImageRegionIterator<OutputImageType> out(output, outputRegion);

  for (in.GoToBegin(), out.GoToBegin(); !in.IsAtEnd() && !out.IsAtEnd(); ++in, ++out)
  {
    out.Set( in.Get() );
  }
}
</pre>

## Continuous integration

* Appveyor (Windows)
* Travis (MacOS)
* CircleCI (Linux)

## Python packages

* Automatically generated in the continuous integration platforms.
* Download wheel to personal computer
* Upload wheel to PyPI (`pip install`).

## Where to find more information:

* ITK Software Guide
    * [Configuring and building ITK](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch2.html#x22-130002)
    * [Create a remote module](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch9.html#x55-1640009.7)
    * [How to write a filter](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch8.html#x54-1330008)
    * [Iterators](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch6.html#x44-1020006)
    * [Modules](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch9.html#x48-1480009)
* [Discourse forum](https://discourse.itk.org/)

## Exercises

### Exercise 1: Create the skeleton of a remote module

* Hint1: You can run command lines by prefixing them with the symbol '!'
* Hint2: You will need to add the argument '--no-input' to the command you are using. This is a limitation due to this notebook environment.

### Exercise 2: Modify the filter

* Add a constant value
* Multiply by a constant factor


### Enjoy ITK!